In [4]:
import numpy as np
import skimage.draw
import tifffile

# Image size and background color
image_size = (2048, 2048)
background_color = 0  # black color in 16-bit

# Initialize a 16-bit image array with the black background
image = np.zeros(image_size, dtype=np.uint16) + background_color

# Eccentricity and centroid of the ellipse
eccentricity = 0.5  # Change this value as needed
centroid_y, centroid_x = 1024, 1024  # Change the centroid position as needed

# Calculate the radii for the major and minor axes
# For a given eccentricity e, the relationship between the axes is: b = a * sqrt(1 - e^2)
# Let's assume the major axis 'a' is half the width of the image
major_axis = image_size[1] // 4
minor_axis = int(major_axis * np.sqrt(1 - eccentricity**2))

# Create an ellipse
rr, cc = skimage.draw.ellipse(centroid_y, centroid_x, minor_axis, major_axis, shape=image.shape)
image[rr, cc] = 65535  # white color in 16-bit

# Save the image as a TIFF file
tiff_file_path = r'C:\Users\Vardan\plaque\data\ecc_test\A04_w2.tif'
tifffile.imsave(tiff_file_path, image)

print(f"Image saved as {tiff_file_path}")


Image saved as C:\Users\Vardan\plaque\data\ecc_test\A04_w2.tif


C:\Users\Vardan\AppData\Local\Temp\ipykernel_49024\4123837570.py:28: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(tiff_file_path, image)


In [6]:
import numpy as np
from PIL import Image, ImageDraw

def circle_safe_to_place(center, radius, existing_circles, image_size):
    # Check if circle is within the image bounds
    if center[0] - radius < 0 or center[1] - radius < 0 or center[0] + radius > image_size[0] or center[1] + radius > image_size[1]:
        return False
    # Check if circle overlaps with existing ones
    return not any(np.hypot(ec[0] - center[0], ec[1] - center[1]) < (ec[2] + radius) for ec in existing_circles)

def place_circle(draw, existing_circles, center, max_radius, min_distance, image_size, color='white'):
    # Attempt to place a single circle within the round shape without overlap
    for _ in range(100):  # max attempts
        distance = np.random.uniform(0, min_distance - max_radius)
        angle = np.random.uniform(0, 2 * np.pi)
        new_center = (center[0] + distance * np.cos(angle), center[1] + distance * np.sin(angle))
        if circle_safe_to_place(new_center, max_radius, existing_circles, image_size):
            new_circle = (new_center[0] - max_radius, new_center[1] - max_radius, 
                          new_center[0] + max_radius, new_center[1] + max_radius)
            draw.ellipse(new_circle, fill=color, outline=color)
            existing_circles.append((new_center[0], new_center[1], max_radius))
            return True
    return False

def draw_round_shape(draw, existing_circles, image_size, num_circles, outer_radius, max_inner_radius):
    # Draw a round shape consisting of non-overlapping circles
    while True:
        # Randomly choose a center for the round shape that stays away from image edges by at least outer_radius
        center = (np.random.uniform(outer_radius, image_size[0] - outer_radius),
                  np.random.uniform(outer_radius, image_size[1] - outer_radius))
        if circle_safe_to_place(center, outer_radius, existing_circles, image_size):
            for _ in range(num_circles):
                if not place_circle(draw, existing_circles, center, max_inner_radius, outer_radius, image_size):
                    break  # Stop trying if a circle can't be placed
            return  # Return after successfully drawing the round shape

def draw_comet(draw, existing_circles, image_size, tail_length, max_radius):
    # Determine start point and orientation for comet
    # The comet head will be placed, then the tail will be constructed with decreasing circle sizes
    while True:
        # Random position for comet head, staying within the bounds considering max_radius as comet head size
        head_center = (np.random.uniform(max_radius, image_size[0] - max_radius),
                       np.random.uniform(max_radius, image_size[1] - max_radius))
        if circle_safe_to_place(head_center, max_radius, existing_circles, image_size):
            existing_circles.append((head_center[0], head_center[1], max_radius))
            break

    # Draw the comet head
    draw.ellipse([head_center[0] - max_radius, head_center[1] - max_radius,
                  head_center[0] + max_radius, head_center[1] + max_radius], fill='white')

    # Draw the comet tail with decreasing sizes
    tail_center = head_center
    for i in range(tail_length):
        tail_radius = max_radius * (0.95 ** i)  # Decrease the radius for each subsequent circle
        if tail_radius < 1:  # If the tail radius is too small, break the loop
            break
        angle_variation = np.random.uniform(-0.1, 0.1)  # Slight angle variation for a wavy tail
        tail_center = (tail_center[0] + tail_radius * np.cos(np.pi + angle_variation), 
                       tail_center[1] + tail_radius * np.sin(np.pi + angle_variation))
        if circle_safe_to_place(tail_center, tail_radius, existing_circles, image_size):
            draw.ellipse([tail_center[0] - tail_radius, tail_center[1] - tail_radius,
                          tail_center[0] + tail_radius, tail_center[1] + tail_radius], fill='white')
            existing_circles.append((tail_center[0], tail_center[1], tail_radius))

def generate_image(image_size, num_round_shapes, num_comets, circles_per_structure, outer_radius, max_inner_radius, tail_length):
    img = Image.new("RGB", image_size, 'black')
    draw = ImageDraw.Draw(img)
    existing_circles = []  # List to keep track of circle centers and radii

    # Draw round shapes
    for _ in range(num_round_shapes):
        draw_round_shape(draw, existing_circles, image_size, circles_per_structure, outer_radius, max_inner_radius)

    # Draw comet shapes
    for _ in range(num_comets):
        draw_comet(draw, existing_circles, image_size, tail_length, max_inner_radius)

    return img

# Parameters
image_size = (2048, 2048)
num_round_shapes = 2  # Number of round shapes to draw
num_comets = 4  # Number of comets to draw
circles_per_structure = 30  # Number of circles in each round shape
outer_radius = 150  # Radius within which to contain the small circles for the round shape
max_inner_radius = 20  # Maximum radius for the small circles and comet head
tail_length = 20  # Length of the comet tail in terms of number of circles

# Generate and show image
image = generate_image(image_size, num_round_shapes, num_comets, circles_per_structure, outer_radius, max_inner_radius, tail_length)
image.show()